In [1]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

import datetime as dt

#https://colab.research.google.com/github/uwdata/visualization-curriculum/blob/master/altair_introduction.ipynb#scrollTo=Zad2hV5G72j2

In [2]:
name = 'TSLA'
# date = dt.date.today()
date = dt.date(2022,5,27)
df = pd.read_csv('./data/{}/{}{}{}'.format(name, name, date,'close'))

chart = alt.Chart(df)



In [3]:
#Option Prices Function

import numpy as np
from scipy import stats
import sys
def optionprice(type,S,K,tau,sig,dft=0):
    '''
    type : option type: 'CALL' or 'PUT'
    S: stock price: float/int
    K: strike price: float/int
    tau: days left to expiration: float/int
    sig: volatility: float/int, without %
    dft: drift rate in years (due to bias or inflation): float, default=0, best match for TSLA around 0.02
    '''
    
    tau = max(tau,0)/ 365 # turn in to years, avoid negative
    sig = sig/100 # add %
    
    if type=='CALL':
        if S == 0:  # this is to avoid log(0) issues
            return 0.0
        elif tau == 0 or sig == 0:  # this is to avoid 0/0 issues
            return max(S - K, 0)
        else:
            d = (np.log(S / K) + dft * tau) / (sig * np.sqrt(tau))
            
            d1 = d + sig * np.sqrt(tau) / 2

            d2 = d - sig * np.sqrt(tau) / 2

            price =  np.exp(dft * tau) * S * stats.norm.cdf(d1, 0.0, 1.0) - K * stats.norm.cdf(d2, 0.0, 1.0)
        return price
        
    elif type == 'PUT':
        if S == 0:  # this is to avoid log(0) issues
            return 0.0
        elif tau == 0 or sig == 0:  # this is to avoid 0/0 issues
            return max(K - S, 0)
        else:
            d = (np.log(S / K) + dft * tau) / (sig * np.sqrt(tau))

            d1 = -d + sig * np.sqrt(tau) / 2

            d2 = -d - sig * np.sqrt(tau) / 2

            price = K * stats.norm.cdf(d1, 0.0, 1.0) - np.exp(dft * tau) * S * stats.norm.cdf(d2, 0.0, 1.0)

            return price
    else:
        sys.exit("option type error")


def optionpriceRow(option,stockprice,date):
    # stockprice: float
    # date: in dt.date or datetime
    
    secondsToExpiration = (option.expirationDate / 1000 - int(date.strftime('%s')))
    daysToExpiration = secondsToExpiration / (24*3600)
    return optionprice(option.putCall,stockprice,option.strikePrice,daysToExpiration,option.volatility)

In [4]:
n = 5

options = df.sample(n)

stockprice = 700

date = dt.date(2022,6,4)

In [5]:


def OptionPriceVis(options,stockprice,date):
    """
    options: dataframe of options (pandas dataframe)
    stockprice: prediction of the underlying stock price (number)
    date: prediction date (dt.date or dt.datetime)
    """

    options['ExpectedPrice'] = options.apply(lambda df: optionpriceRow(df,stockprice,date),axis=1)


    options['Num'] = options.apply(lambda df: 1,axis=1)


    expected = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='ExpectedPrice:Q',
        tooltip = ['description','ExpectedPrice','mark'],
    )

    putCall_color = alt.Color('putCall:N',scale=alt.Scale(domain=['PUT','CALL'],range=['red','green']))


    mark = alt.Chart(options).mark_tick(thickness=3).encode(
        y='symbol:N',
        x='mark:Q',
        color = putCall_color
    )

    return mark + expected

OptionPriceVis(options,stockprice,date)

alt.LayerChart(...)

In [6]:
# PnL diagram


centerPrice = 750


def Option_PnL_Vis(options,date,centerPrice):
    """
    options: dataframe of options (pandas dataframe)
    date: date of PnL  (dt.date or dt.datetime)
    centerPrice: The visualization will draw the PnL diagram around centerPrice
    """

    def option_stockprice_DF(centerPrice,options,stepNum=500,percent=0.8):
        # https://stackoverflow.com/questions/42168103/how-to-expand-flatten-pandas-dataframe-efficiently
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html

        """
        Create a dataframe of options with varying stockPrice
        """
        low = centerPrice*((1-percent))
        high = centerPrice*((1+percent))
        step = (high-low) / stepNum
        priceRange = [ low + step*num for num in range(stepNum+1)]
        options['stockPrice'] = options.apply(lambda df:priceRange,axis=1)
        newdf = options.explode('stockPrice')
        return newdf


    optionss = option_stockprice_DF(centerPrice,options)

    optionss['ExpectedPrice'] = optionss.apply(lambda df: optionpriceRow(df,df.stockPrice,date),axis=1)

    optionss['Num'] = options.apply(lambda df: 1,axis=1) # number of options for the position

    optionss['Return'] = optionss.apply(lambda df: (df.ExpectedPrice - df.mark)*df.Num,axis=1)

    indivual = alt.Chart(optionss).mark_line(thickness=20).encode(
        x='stockPrice:Q',
        y='Return:Q',
        color='symbol:N',
        tooltip = ['description','Return','Num']
    )

    total = alt.Chart(optionss).mark_line(color='black',thickness=40).encode(
        x='stockPrice:Q',
        y='sum(Return):Q',
    )

    res = indivual + total

    res = alt.concat(res).properties(
        title=alt.TitleParams(
            ['black line is the overall PnL'],
            baseline='bottom',
            orient='bottom',
            anchor='end',
            fontWeight='normal',
            fontSize=10,

            )
    )

    return res

Option_PnL_Vis(options,date,centerPrice)



alt.ConcatChart(...)

In [7]:
# Greeks Vis


def Greeks_Vis(options,stockprice,date):
    """
    options: dataframe of options (pandas dataframe)
    stockprice: prediction of the underlying stock price (number)
    date: prediction date (dt.date or dt.datetime)
    """


    def daysLeft(expiration,date):
        #  exipration: in Epoch ms
        # date: in dt.date or datetime
        secondsToExpiration = (expiration / 1000 - int(date.strftime('%s')))
        daysToExpiration = secondsToExpiration / (24*3600)
        return daysToExpiration

    def greeks_daily(name,type,S,K,tau,sig,dft=0):
        if name=='delta':
            res = optionprice(type,S,K,tau,sig,dft)-optionprice(type,S+1,K,tau,sig,dft)
        elif name=='theta':
            res = optionprice(type,S,K,tau,sig,dft)-optionprice(type,S,K,tau+1,sig,dft)
        else:
            sys.exit("option type error")
        return res


    options['Num'] = options.apply(lambda df: 1,axis=1)

    options['delta_daily'] = options.apply(lambda df: greeks_daily('delta',df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,date),df.volatility),axis=1)
    options['theta_daily'] = options.apply(lambda df: greeks_daily('theta',df.putCall,stockprice,df.strikePrice,daysLeft(df.expirationDate,date),df.volatility),axis=1)


    delta_vis = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='delta:Q',
        tooltip = ['description','delta'],
    )

    theta_vis = alt.Chart(options).mark_bar(opacity=0.5).encode(
        y='symbol:N',
        x='theta:Q',
        tooltip = ['description','theta'],
    )

    return delta_vis & theta_vis

Greeks_Vis(options,stockprice,date)

alt.VConcatChart(...)